<a href="https://colab.research.google.com/github/vptuan/COMP1804AML/blob/master/01_intro_api_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[![AnalyticsDojo](https://github.com/rpi-techfundamentals/spring2019-materials/blob/master/fig/final-logo.png?raw=1)](http://rpi.analyticsdojo.com)
<center><h1>Introduction to API's with Python</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>



This is adopted from [Mining the Social Web, 2nd Edition](http://bit.ly/16kGNyb)
Copyright (c) 2013, Matthew A. Russell
All rights reserved.

This work is licensed under the [Simplified BSD License](https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/LICENSE.txt).

### Before you Begin #1
If you are working locally or on colab, this exercise requires the twitter package and the ruamel.yaml package.   Yaml files are structured files useful for storing configuration. 


In [0]:
 !pip install twitter ruamel.yaml

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 133kB 13.1MB/s 
     |████████████████████████████████| 552kB 19.2MB/s 


In [0]:
#see if it worked by importing the twitter package & some other things we will use.  
from  twitter import *
import datetime, traceback 
import json
import time
import sys


### Before you Begin #2
Download the sample configuration.  


In [0]:
!!wget https://raw.githubusercontent.com/rpi-techfundamentals/spring2019-materials/master/04-viz-api-scraper/screen_names.csv && wget https://raw.githubusercontent.com/rpi-techfundamentals/spring2019-materials/master/04-viz-api-scraper/twitlab.py 


['--2020-01-31 03:34:04--  https://raw.githubusercontent.com/rpi-techfundamentals/spring2019-materials/master/04-viz-api-scraper/screen_names.csv',
 'Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...',
 'Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: 63 [text/plain]',
 'Saving to: ‘screen_names.csv’',
 '',
 '',
 'screen_names.csv      0%[                    ]       0  --.-KB/s               ',
 'screen_names.csv    100%[===================>]      63  --.-KB/s    in 0s      ',
 '',
 '2020-01-31 03:34:04 (15.4 MB/s) - ‘screen_names.csv’ saved [63/63]',
 '',
 '--2020-01-31 03:34:04--  https://raw.githubusercontent.com/rpi-techfundamentals/spring2019-materials/master/04-viz-api-scraper/twitlab.py',
 'Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.12

# Download Authorization file (look at Slack for file)

In [0]:
!wget https://www.dropbox.com/s/ojjs2aj14gjuqjb/config.yaml 

--2020-01-31 03:34:14--  https://www.dropbox.com/s/ojjs2aj14gjuqjb/config.yaml
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ojjs2aj14gjuqjb/config.yaml [following]
--2020-01-31 03:34:15--  https://www.dropbox.com/s/raw/ojjs2aj14gjuqjb/config.yaml
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7b8b86fa109f6f62ba8b229027.dl.dropboxusercontent.com/cd/0/inline/AxN1sGuLXjQLFqKcbwT2KW5uK5pO6dGQ_o9fg4_OQbpiGQPzNym4l9aIzEuVANW-mVCzCmOeEAWexBNS21S7WqEIJKtjYjQ4HMJ7NSGehuCqMg/file# [following]
--2020-01-31 03:34:15--  https://uc7b8b86fa109f6f62ba8b229027.dl.dropboxusercontent.com/cd/0/inline/AxN1sGuLXjQLFqKcbwT2KW5uK5pO6dGQ_o9fg4_OQbpiGQPzNym4l9aIzEuVANW-mVCzCmOeEAWexBNS21S7WqEIJKtjYjQ4HMJ7NSGehuCqMg/file
Resolving uc7b8b86f

## Step1.  Loading Authorization Data
- Here we are going to store the authorization data in a .YAML file rather than directly in the notebook.  
- We have also added `config.yaml` to the `.gitignore` file so we won't accidentally commit our sensitive data to the repository.
- You should generally keep sensitive data out of all git repositories (public or private) but definitely Public. 
- If you ever accidentally commit data to a public repository you must consider it compromised.
- A .yaml file is a common way to store configuration data, but it is not really secure. 

In [0]:
#This will import some required libraries.
import sys 
import ruamel.yaml #A .yaml file 
#This is your configuration file. 
twitter_yaml='config.yaml'
with open(twitter_yaml, 'r') as yaml_t:
    cf_t=ruamel.yaml.round_trip_load(yaml_t, preserve_quotes=True)

#You can check your config was loaded by printing, but you should not commit this.
cf_t


CommentedMap([('file', 'screen_names.csv'),
              ('config', './twitter'),
              ('data', './data'),
              ('consumer_key', '51vEg79kMf7J0unZ9M0PICtxl'),
              ('consumer_secret',
               'Fi3uYkIUMBsRUt5EoNJXVHKdlWVY4opgGbh08LV0up175qKlC6'),
              ('access_token',
               '18250462-PlNse0CB5ZY1vcZq6tOdLWS6UBp6p6SUUObdjoFcB'),
              ('access_token_secret',
               'CcofKfnl1ayVyLiTrANtaOP2ZeuQVzoVHeYZhIV1EOha2'),
              ('sleep_interval', 5),
              ('twitter_cutoff', 3195)])

# `cat` command to look at files
We can use the `cat` command to look at the structure of our files. 

In [0]:
!cat config.yaml

file: screen_names.csv  #Filename for list of twitter IDs.
config: ./twitter  #Spot for the configuration.
data: ./data      #Filename for data.
consumer_key: 51vEg79kMf7J0unZ9M0PICtxl
consumer_secret: Fi3uYkIUMBsRUt5EoNJXVHKdlWVY4opgGbh08LV0up175qKlC6
access_token: 18250462-PlNse0CB5ZY1vcZq6tOdLWS6UBp6p6SUUObdjoFcB
access_token_secret: CcofKfnl1ayVyLiTrANtaOP2ZeuQVzoVHeYZhIV1EOha2
sleep_interval: 5 ## twitter api access limit Requests / 15-min window (user auth) 180 => 5 sec/request
twitter_cutoff: 3195 #Twitter cuts off at 3195 tweets per user.


## Create Some Relevant Functions
- We first will create a Twitter object we can used to authorize data.
- Then we will get profiles.
- Finally we will get some tweets.  

**Don't worry about not understanding all the code.  Here we are pushing you us more complex functions.**

In [0]:
#@title
def create_twitter_auth(cf_t):
        """Function to create a twitter object
           Args: cf_t is configuration dictionary. 
           Returns: Twitter object.
            """
        # When using twitter stream you must authorize.
        # these tokens are necessary for user authentication
        # create twitter API object

        auth = OAuth(cf_t['access_token'], cf_t['access_token_secret'], cf_t['consumer_key'], cf_t['consumer_secret'])

        try:
            # create twitter API object
            twitter = Twitter(auth = auth)
        except TwitterHTTPError:
            traceback.print_exc()
            time.sleep(cf_t['sleep_interval'])
        return twitter

In [0]:
def get_profiles(twitter, names, cf_t):
    """Function write profiles to a file with the form *data-user-profiles.json*
       Args: names is a list of names
             cf_t is a list of twitter config
       Returns: Nothing
        """
    # file name for daily tracking
    dt = datetime.datetime.now()
    fn = cf_t['data']+'/profiles/'+dt.strftime('%Y-%m-%d-user-profiles.json')
    with open(fn, 'w') as f:
        for name in names:
            print("Searching twitter for User profile: ", name)
            try:
                # create a subquery, looking up information about these users
                # twitter API docs: https://dev.twitter.com/docs/api/1/get/users/lookup
                profiles = twitter.users.lookup(screen_name = name)
                sub_start_time = time.time()
                for profile in profiles:
                    print("User found. Total tweets:", profile['statuses_count'])
                    # now save user info
                    f.write(json.dumps(profile))
                    f.write("\n")
                sub_elapsed_time = time.time() - sub_start_time;
                if sub_elapsed_time < cf_t['sleep_interval']:
                    time.sleep(cf_t['sleep_interval'] + 1 - sub_elapsed_time)
            except TwitterHTTPError:
                traceback.print_exc()
                time.sleep(cf_t['sleep_interval'])
                continue
    f.close()
    return fn

## Load Twitter Handle From CSV
- This is a .csv that has individuals we want to collect data on. 
- Go ahead and follow [AnalyticsDojo](https://twitter.com/AnalyticsDojo).  

In [0]:
import pandas as pd
df=pd.read_csv(cf_t['file'])
df

,index,screen_name
0,1,tensorflow
1,2,DeepLearningHub
2,3,neiltyson


## Create Twitter Object

In [0]:
import twitlab

In [0]:
#Create Twitter Object
twitter= twitlab.create_twitter_auth(cf_t)

In [0]:
df['screen_name']

0         tensorflow
1    DeepLearningHub
2          neiltyson
Name: screen_name, dtype: object

In [0]:
!mkdir data && mkdir data/profiles

In [0]:
#This will get general profile data
profiles_fn=twitlab.get_profiles(twitter, df['screen_name'], cf_t)

Searching twitter for User profile:  tensorflow
User found. Total tweets: 977
Searching twitter for User profile:  DeepLearningHub
User found. Total tweets: 2395
Searching twitter for User profile:  neiltyson
User found. Total tweets: 6387


In [0]:
!ls data/profiles

# Let's look at the files created.  

In [0]:
!cat data/profiles/*-user-profiles.json

{"id": 254107028, "id_str": "254107028", "name": "TensorFlow", "screen_name": "TensorFlow", "location": "Mountain View, CA", "description": "TensorFlow is a fast, flexible, and scalable open-source machine learning library for research and production.", "url": "https://t.co/gxgTGHp5yO", "entities": {"url": {"urls": [{"url": "https://t.co/gxgTGHp5yO", "expanded_url": "http://tensorflow.org", "display_url": "tensorflow.org", "indices": [0, 23]}]}, "description": {"urls": []}}, "protected": false, "followers_count": 196870, "friends_count": 96, "listed_count": 2130, "created_at": "Fri Feb 18 16:21:31 +0000 2011", "favourites_count": 185, "utc_offset": null, "time_zone": null, "geo_enabled": true, "verified": true, "statuses_count": 977, "lang": null, "status": {"created_at": "Thu Jan 30 20:47:20 +0000 2020", "id": 1222984664471588866, "id_str": "1222984664471588866", "text": "Train with the TensorFlow team! \ud83c\udfcb\ufe0f\u200d\u2642\ufe0f\n\nIn this session of Inside TensorFlow, Soft

The outcoming of running the above API is to generate a twitter object. 

## Step 2. Getting Help

In [0]:
# We can get some help on how to use the twitter api with the following. 
help(twitter)

Help on Twitter in module twitter.api object:

class Twitter(TwitterCall)
 |  The minimalist yet fully featured Twitter API class.
 |  
 |  Get RESTful data by accessing members of this class. The result
 |  is decoded python objects (lists and dicts).
 |  
 |  The Twitter API is documented at:
 |  
 |    https://dev.twitter.com/overview/documentation
 |  
 |  The list of most accessible functions is listed at:
 |  
 |    https://dev.twitter.com/rest/public
 |  
 |  
 |  Examples::
 |  
 |      from twitter import *
 |  
 |      t = Twitter(
 |          auth=OAuth(token, token_secret, consumer_key, consumer_secret))
 |  
 |      # Get your "home" timeline
 |      t.statuses.home_timeline()
 |  
 |      # Get a particular friend's timeline
 |      t.statuses.user_timeline(screen_name="billybob")
 |  
 |      # to pass in GET/POST parameters, such as `count`
 |      t.statuses.home_timeline(count=5)
 |  
 |      # to pass in the GET/POST parameter `id` you need to use `_id`
 |      t.sta


Go ahead and take a look at the [twitter docs](https://dev.twitter.com/rest/public).



In [0]:
# The Yahoo! Where On Earth ID for the entire world is 1.
# See https://dev.twitter.com/docs/api/1.1/get/trends/place and
# http://developer.yahoo.com/geo/geoplanet/

WORLD_WOE_ID = 1
US_WOE_ID = 23424977

# Prefix ID with the underscore for query string parameterization.
# Without the underscore, the twitter package appends the ID value
# to the URL itself as a special case keyword argument.

world_trends = twitter.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter.trends.place(_id=US_WOE_ID)

print (world_trends)
print (us_trends)

[{'trends': [{'name': 'Petrix', 'url': 'http://twitter.com/search?q=Petrix', 'promoted_content': None, 'query': 'Petrix', 'tweet_volume': 634551}, {'name': '#Walls', 'url': 'http://twitter.com/search?q=%23Walls', 'promoted_content': None, 'query': '%23Walls', 'tweet_volume': 519325}, {'name': '#TheGoodPlace', 'url': 'http://twitter.com/search?q=%23TheGoodPlace', 'promoted_content': None, 'query': '%23TheGoodPlace', 'tweet_volume': 34889}, {'name': '#나는_이노래로_입덕했다', 'url': 'http://twitter.com/search?q=%23%EB%82%98%EB%8A%94_%EC%9D%B4%EB%85%B8%EB%9E%98%EB%A1%9C_%EC%9E%85%EB%8D%95%ED%96%88%EB%8B%A4', 'promoted_content': None, 'query': '%23%EB%82%98%EB%8A%94_%EC%9D%B4%EB%85%B8%EB%9E%98%EB%A1%9C_%EC%9E%85%EB%8D%95%ED%96%88%EB%8B%A4', 'tweet_volume': 12012}, {'name': '#chilexel13', 'url': 'http://twitter.com/search?q=%23chilexel13', 'promoted_content': None, 'query': '%23chilexel13', 'tweet_volume': None}, {'name': 'Devin Booker', 'url': 'http://twitter.com/search?q=%22Devin+Booker%22', 'promo

## Step 3. Displaying API responses as pretty-printed JSON

In [0]:
import json

print (json.dumps(world_trends, indent=1))
print (json.dumps(us_trends, indent=1))

[
 {
  "trends": [
   {
    "name": "Petrix",
    "url": "http://twitter.com/search?q=Petrix",
    "promoted_content": null,
    "query": "Petrix",
    "tweet_volume": 634551
   },
   {
    "name": "#Walls",
    "url": "http://twitter.com/search?q=%23Walls",
    "promoted_content": null,
    "query": "%23Walls",
    "tweet_volume": 519325
   },
   {
    "name": "#TheGoodPlace",
    "url": "http://twitter.com/search?q=%23TheGoodPlace",
    "promoted_content": null,
    "query": "%23TheGoodPlace",
    "tweet_volume": 34889
   },
   {
    "name": "#\ub098\ub294_\uc774\ub178\ub798\ub85c_\uc785\ub355\ud588\ub2e4",
    "url": "http://twitter.com/search?q=%23%EB%82%98%EB%8A%94_%EC%9D%B4%EB%85%B8%EB%9E%98%EB%A1%9C_%EC%9E%85%EB%8D%95%ED%96%88%EB%8B%A4",
    "promoted_content": null,
    "query": "%23%EB%82%98%EB%8A%94_%EC%9D%B4%EB%85%B8%EB%9E%98%EB%A1%9C_%EC%9E%85%EB%8D%95%ED%96%88%EB%8B%A4",
    "tweet_volume": 12012
   },
   {
    "name": "#chilexel13",
    "url": "http://twitter.com/search?q

Take a look at the [api docs](https://dev.twitter.com/rest/reference/get/trends/place) for the /trends/place call made above. 

## Step 4. Collecting search results for a targeted hashtag.

In [0]:
# Import unquote to prevent url encoding errors in next_results
#from urllib3 import unquote

#This can be any trending topic, but let's focus on a hashtag that is relevant to the class. 
q = '#analytics' 

count = 100

# See https://dev.twitter.com/rest/reference/get/search/tweets
search_results = twitter.search.tweets(q=q, count=count)

#This selects out 
statuses = search_results['statuses']


# Iterate through 5 more batches of results by following the cursor
for _ in range(5):
    print ("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
        print ("next_results", next_results)
    except: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    print (kwargs)
    search_results = twitter.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
print (json.dumps(statuses[0], indent=1))

Length of statuses 100
next_results ?max_id=1223073890047533055&q=%23analytics&count=100&include_entities=1
{'max_id': '1223073890047533055', 'q': '%23analytics', 'count': '100', 'include_entities': '1'}
Length of statuses 200
next_results ?max_id=1223059115381075969&q=%2523analytics&count=100&include_entities=1
{'max_id': '1223059115381075969', 'q': '%2523analytics', 'count': '100', 'include_entities': '1'}
Length of statuses 200
{
 "created_at": "Fri Jan 31 03:36:11 +0000 2020",
 "id": 1223087555635302400,
 "id_str": "1223087555635302400",
 "text": "RT @gp_pulipaka: Radiant \u2013 #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT\u2026",
 "truncated": false,
 "entities": {
  "hashtags": [
   {
    "text": "BusinessAnalytics",
    "indices": [
     27,
     45
    ]
   },
   {
    "text": "Shiny",
    "indices": [
     58,
     64
    ]
   },
   {
    "text": "BigData",
    "indices": [
     66,
     74
    ]
   },
   {
  

In [0]:
#Print several
print (json.dumps(statuses[0:5], indent=1))

[
 {
  "created_at": "Fri Jan 31 03:36:11 +0000 2020",
  "id": 1223087555635302400,
  "id_str": "1223087555635302400",
  "text": "RT @gp_pulipaka: Radiant \u2013 #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT\u2026",
  "truncated": false,
  "entities": {
   "hashtags": [
    {
     "text": "BusinessAnalytics",
     "indices": [
      27,
      45
     ]
    },
    {
     "text": "Shiny",
     "indices": [
      58,
      64
     ]
    },
    {
     "text": "BigData",
     "indices": [
      66,
      74
     ]
    },
    {
     "text": "Analytics",
     "indices": [
      75,
      85
     ]
    },
    {
     "text": "DataScience",
     "indices": [
      86,
      98
     ]
    },
    {
     "text": "AI",
     "indices": [
      99,
      102
     ]
    },
    {
     "text": "MachineLearning",
     "indices": [
      103,
      119
     ]
    },
    {
     "text": "Jupyter",
     "indices": [
      120,
      128
     ]

## Step 5. Extracting text, screen names, and hashtags from tweets

In [0]:
#We can access an individual tweet like so:
statuses[1]['text']





'RT @gp_pulipaka: Radiant – #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT…'

In [0]:
statuses[1]['entities']

{'hashtags': [{'indices': [27, 45], 'text': 'BusinessAnalytics'},
  {'indices': [58, 64], 'text': 'Shiny'},
  {'indices': [66, 74], 'text': 'BigData'},
  {'indices': [75, 85], 'text': 'Analytics'},
  {'indices': [86, 98], 'text': 'DataScience'},
  {'indices': [99, 102], 'text': 'AI'},
  {'indices': [103, 119], 'text': 'MachineLearning'},
  {'indices': [120, 128], 'text': 'Jupyter'},
  {'indices': [129, 133], 'text': 'IoT'},
  {'indices': [134, 139], 'text': 'IIoT'}],
 'symbols': [],
 'urls': [],
 'user_mentions': [{'id': 4263007693,
   'id_str': '4263007693',
   'indices': [3, 15],
   'name': 'Dr. GP Pulipaka',
   'screen_name': 'gp_pulipaka'}]}

In [0]:
#notice the nested relationships.  We have to take notice of this to further access the data.
statuses[1]['entities']['hashtags']

[{'indices': [27, 45], 'text': 'BusinessAnalytics'},
 {'indices': [58, 64], 'text': 'Shiny'},
 {'indices': [66, 74], 'text': 'BigData'},
 {'indices': [75, 85], 'text': 'Analytics'},
 {'indices': [86, 98], 'text': 'DataScience'},
 {'indices': [99, 102], 'text': 'AI'},
 {'indices': [103, 119], 'text': 'MachineLearning'},
 {'indices': [120, 128], 'text': 'Jupyter'},
 {'indices': [129, 133], 'text': 'IoT'},
 {'indices': [134, 139], 'text': 'IIoT'}]

In [0]:
status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

urls = [ url['url'] 
             for status in statuses
                 for url in status['entities']['urls'] ]



# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

# Explore the first 5 items for each...

print (json.dumps(status_texts[0:5], indent=1))
print (json.dumps(screen_names[0:5], indent=1)) 
print (json.dumps(hashtags[0:5], indent=1))
print (json.dumps(words[0:5], indent=1))

[
 "RT @gp_pulipaka: Radiant \u2013 #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT\u2026",
 "RT @gp_pulipaka: Radiant \u2013 #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT\u2026",
 "RT @RyanGlass4reals: #Entreprenuer: Ad Exec on Using 'Creative Ninjutsu' Tactics for Entrepreneurs https://t.co/oH8Pu6Uq2D #startup #growth\u2026",
 "RT @gp_pulipaka: Radiant \u2013 #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT\u2026",
 "RT @gp_pulipaka: Radiant \u2013 #BusinessAnalytics using R and #Shiny. #BigData #Analytics #DataScience #AI #MachineLearning #Jupyter #IoT #IIoT\u2026"
]
[
 "gp_pulipaka",
 "gp_pulipaka",
 "RyanGlass4reals",
 "gp_pulipaka",
 "gp_pulipaka"
]
[
 "BusinessAnalytics",
 "Shiny",
 "BigData",
 "Analytics",
 "DataScience"
]
[
 "RT",
 "@gp_pulipaka:",
 "Radiant",
 "\u2013",
 "#BusinessAn

## Step 6. Creating a basic frequency distribution from the words in tweets

In [0]:
from collections import Counter

for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print (c.most_common()[:10]) # top 10, "\n")
    

[('RT', 151), ('#AI', 107), ('#Analytics', 98), ('#BigData', 86), ('#DataScience', 85), ('#MachineLearning', 84), ('#IoT', 82), ('@gp_pulipaka:', 76), ('and', 39), ('to', 36)]
[('gp_pulipaka', 76), ('ccplus', 12), ('RyanGlass4reals', 9), ('digitalked', 8), ('thestartup_', 7), ('Forbes', 7), ('koshyaby27', 6), ('davidprosserind', 6), ('Ronald_vanLoon', 5), ('GavLaaaaaaaa', 3)]
[('AI', 116), ('Analytics', 101), ('BigData', 93), ('DataScience', 86), ('MachineLearning', 85), ('IoT', 83), ('IIoT', 48), ('Kubernetes', 18), ('analytics', 15), ('ArtificialIntelligence', 12)]
